### Importación de los módulos necesarios

In [32]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from pmdarima.arima import ndiffs, nsdiffs
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, r2_score
from scipy.stats import uniform
import seaborn as sns
from dateutil.relativedelta import relativedelta
from metricas import calculo_metricas, plot_meses

import warnings
warnings.simplefilter("ignore")

# Mostramos las versiones de los módulos para posibles reproducciones del código

print('Versión pandas:', pd.__version__)
print('Versión numpy:', np.__version__)
print('Versión matplotlib:', matplotlib.__version__)


Versión pandas: 1.0.5
Versión numpy: 1.19.5
Versión matplotlib: 3.2.2


### Carga del dataframe

In [33]:
df = pd.read_csv('Data/dataframe.csv', index_col = 0)
df.index = pd.to_datetime(df.index)
df = df.drop(columns = ["Festivo_Regional", "Humedad_Relativa", "Precipitacion", "Radiacion",
                        "Velocidad_Viento",'Precio_max', 'Precio_min','Spred_precio', 'Temperatura',
                        'Temperatura_max', 'Temperatura_min','Spred_temperatura'])

df.loc[:, "lag_24"] = df.Spot_electricidad.shift(24)
df.loc[:, "lag_48"] = df.Spot_electricidad.shift(48)
df.loc[:, "lag_1_semana"] = df.Spot_electricidad.shift(24*7)
db = df.copy()

### Parameters:

- n_estimators=100
- criterion='mse'
- max_depth=None
- min_samples_split=2
- min_samples_leaf=1
- min_weight_fraction_leaf=0.0
- max_features='auto'
- max_leaf_nodes=None
- min_impurity_decrease=0.0
- min_impurity_split=None
- bootstrap=True
- oob_score=False
- n_jobs=None
- random_state=None
- verbose=0
- warm_start=False
- ccp_alpha=0.0
- max_samples=None

In [27]:
db.columns

Index(['Spot_electricidad', 'Prevision_Demanda', 'Prevision_Eol_Fotov',
       'Hueco_Termico', 'Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes',
       'Sabado', 'Domingo', 'Hora_1', 'Hora_2', 'Hora_3', 'Hora_4', 'Hora_5',
       'Hora_6', 'Hora_7', 'Hora_8', 'Hora_9', 'Hora_10', 'Hora_11', 'Hora_12',
       'Hora_13', 'Hora_14', 'Hora_15', 'Hora_16', 'Hora_17', 'Hora_18',
       'Hora_19', 'Hora_20', 'Hora_21', 'Hora_22', 'Hora_23', 'Hora_24',
       'Festivo_Nacional', 'MIBGAS_FW1D', 'Brent', 'Api2', 'IBEX', 'lag_24',
       'lag_48', 'lag_1_semana'],
      dtype='object')

In [34]:
def bascktesting_random_forest(db, año, mes, rolling_window, days_start, target_mean = False):
    
    """Función que realiza un backtesting del modelo de Random Forest.
    Parametros:
        - db: dataframe con la variable a explicar y las variables explicativas
        - año: año desde el que se quiere comenzar a realizar el backtesting
        - mes: mes desde el que se quiere comenzar a realizar el backtesting
        - rolling_window: valor booleano que describe si quieres ventana de entrenamiento móvil o no
        - days_start: en caso de que rolling window sea True, coge los días de la ventana móvil fija de entrenamiento
        - target_mean: valor booleano que indica si realizar la técnica target mean o no
    output:
        - array_pred: vector de valores de predicción
        - array_real: vector de valores reales con los que se compara la predicción
        - array_date: fechas horarias a las que corresponde los valores reales y predichos
    
    """
    
    fecha = datetime(año, mes, 1)
    fecha_str = str(año) + '-' + str(mes) + '-' + str(1)
    
    db = db.dropna()
    array_real, array_pred, array_date = np.zeros(len(db.loc[fecha_str:])), np.zeros(len(db.loc[fecha_str:])), []

    # En caso de que haya rolling window
    if rolling_window == True:
        for day in range(int(len(db.loc[fecha_str:]) / 24)):
            print(day)
            fecha_end_train = fecha + relativedelta(days = day)
            fecha_start_train = fecha_end_train - relativedelta(days = days_start)
            fecha_end_train = str(fecha_end_train.year) + "-" + str(fecha_end_train.month) + "-" + str(fecha_end_train.day)
            fecha_start_train = str(fecha_start_train.year) + "-" + str(fecha_start_train.month) + "-" + str(fecha_start_train.day)
            db_aux = db.loc[fecha_start_train:fecha_end_train]
            
            # Dataframe de Entrenamiento
            X_train = db_aux
            
            
            # Dataframe de Test
            X_test = X_train.loc[fecha_end_train]
            

            # Quitamos al dataframe de train, el dataframe de test
            X_train = X_train.drop(index = X_test.index)
            
            # Target mean
            if target_mean:
                
                # Si target_mean es True, se eliminan las variables dummies de train y test, ya que van a entrar de otra forma
                
                X_train = X_train.drop(columns = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes',
                                               'Sabado', 'Domingo', 'Hora_1', 'Hora_2', 'Hora_3', 'Hora_4', 'Hora_5',
                                               'Hora_6', 'Hora_7', 'Hora_8', 'Hora_9', 'Hora_10', 'Hora_11', 'Hora_12',
                                               'Hora_13', 'Hora_14', 'Hora_15', 'Hora_16', 'Hora_17', 'Hora_18',
                                               'Hora_19', 'Hora_20', 'Hora_21', 'Hora_22', 'Hora_23', 'Hora_24'])
                
                X_test = X_test.drop(columns = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes',
                                               'Sabado', 'Domingo', 'Hora_1', 'Hora_2', 'Hora_3', 'Hora_4', 'Hora_5',
                                               'Hora_6', 'Hora_7', 'Hora_8', 'Hora_9', 'Hora_10', 'Hora_11', 'Hora_12',
                                               'Hora_13', 'Hora_14', 'Hora_15', 'Hora_16', 'Hora_17', 'Hora_18',
                                               'Hora_19', 'Hora_20', 'Hora_21', 'Hora_22', 'Hora_23', 'Hora_24'])
                
                # Guardamos los datos 'encoded' en un diccionario para luego construir la columna
                dic_hora , dic_dia = {} , {}
                
                # Creamos el diccionario para las horas codificadas
                for i in range(24):
                    dic_hora[X_train.groupby([X_train.index.hour])[["Spot_electricidad"]].mean().reset_index().iloc[i,0]] = \
                             X_train.groupby([X_train.index.hour])[["Spot_electricidad"]].mean().reset_index().iloc[i,1]
                    
                # Hour encoded para train y test
                X_train.loc[:, "Hour_encoded"] = [dic_hora[dato] for dato in X_train.index.hour]
                X_test.loc[:, "Hour_encoded"] = [dic_hora[dato] for dato in X_test.index.hour]
                             
                # Creamos el diccionario para los días codificados
                for i in range(7):
                    dic_dia[X_train.groupby([X_train.index.weekday])[["Spot_electricidad"]].mean().reset_index().iloc[i,0]] = \
                            X_train.groupby([X_train.index.weekday])[["Spot_electricidad"]].mean().reset_index().iloc[i,1]
                    
                # Creamos las columnas finales
                X_train.loc[:, "Day_encoded"] = [dic_dia[dato] for dato in X_train.index.weekday]
                X_test.loc[:, "Day_encoded"] = [dic_dia[dato] for dato in X_test.index.weekday]
                    
            # Quitamos la variable objetivo
            y_train = X_train.pop('Spot_electricidad')
            y_test = X_test.pop('Spot_electricidad')

            # Entrenamos el modelo
            model = RandomForestRegressor(n_estimators = 150,
                              criterion = "mae",
                              max_depth = None,
                              max_features = X_train.shape[1]- 6,
                              n_jobs = -1,
                              warm_start = False,
                              random_state = 123).fit(X_train,y_train)
            
            # Realizamos las predicciones
            pred = model.predict(X_test)
            date = X_test.index

            array_date.append(date)
            array_pred[day*24:day*24 + 24] = pred
            array_real[day*24:day*24 + 24] = y_test
        
    # En caso de que no haya rolling window, siempre cogemos todo el dataframe disponible
    else:
        for day in range(int(len(db.loc[fecha_str:]) / 24)):
            print(day)
            fecha_end_train = fecha + relativedelta(days = day)
            fecha_end_train = str(fecha_end_train.year) + "-" + str(fecha_end_train.month) + "-" + str(fecha_end_train.day)
            db_aux = db.loc[:fecha_end_train]

            # Dataframe de Entrenamiento
            X_train = db_aux
            y_train = X_train.pop('Spot_electricidad')
            
            # Dataframe de Test
            X_test = X_train.loc[fecha_end_train]
            y_test = y_train.pop(fecha_end_train)
            
            # Quitamos al dataframe de train, el dataframe de test
            X_train = X_train.drop(index = X_train.iloc[-24:].index)
            
            # Entrenamos el modelo
            model = RandomForestRegressor(n_estimators = 100,
                              criterion = "mae",
                              max_depth = None,
                              max_features = X_train.shape[1] - 1,
#                               n_jobs = -1,
                              random_state = 123).fit(X_train,y_train)
            
            # Realizamos las predicciones
            pred = model.predict(X_test)
            date = X_test.index

            array_date.append(date)
            array_pred[day*24:day*24 + 24] = pred
            array_real[day*24:day*24 + 24] = y_test
            
    array_date = [item for sublist in array_date for item in sublist]
    return array_pred, array_real, array_date

pred, real, dates = bascktesting_random_forest(db.loc["2019":], 2020, 6,
                                               rolling_window = True, days_start = 30, target_mean = True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Calculo de las metricas

In [36]:
resultados = pd.DataFrame({"Pred":pred, "Real":real}, index = dates)
resultados["Pred"][resultados.loc[:, "Pred"] < 0] = 0

rf, rf_metricas_mensuales, met = calculo_metricas(resultados)

     MAE  MAE (median)   MAPE  WMAPE   RMSE  % Trend
0  10.02          6.19  74.47   8.95  16.34    82.29


In [40]:
# Exportamos los resultados de la mejor combinación
rf.to_csv('Resultados/resultados_rf.csv')
rf_metricas_mensuales.to_csv('Resultados/resultados_mensuales_rf.csv')

In [16]:
# Visualizamos el backtesting
plot_meses(rf)